In [5]:
import pandas as pd
import csv
from demyst.analytics import Analytics
from collections import Counter

!pip3 install ukpc

from ukpc import PostCode
analytics = Analytics()

## Read In Your Input Data File

In [16]:
inputs = pd.read_csv('https://s3.amazonaws.com/demyst-apis/demos/all_reg_no.csv', header = None)
inputs.columns = ['company_number']
inputs['client_id'] =  inputs.index

## List All Of The Relevant Providers For This Use Case

In [17]:
all_providers = ['acuris_business_search', 
                 'companies_house_company_officers',
                 'companies_house_company_profile',
                 'duedil_company_charges', 
                 'duedil_company_group_parents',
                 'duedil_company_group_subsidiaries',
                 'duedil_company_officers',
                 'duedil_company_related_companies',
                 'duedil_company_vitals',
                 'duedil_find_company',
                 'globaldatabase_company_details',
                 'globaldatabase_find_companies',
                 'matchdeck_search',
                 'matchdeck_details',
                 'opencorporates_companies_search',
                 'orb_search',
                 'owler_company_premium',
                 'pipl_premium_search']

## Get Company Name and Address through Companies House

In [20]:
providers = ['companies_house_company_profile', 'companies_house_company_officers']
company_basic = analytics.enrich_and_download(providers, inputs[0:1], validate=False)
company_basic.rename(columns = {'companies_house_company_profile.client_id': 'client_id',
                                'inputs.company_number': 'company_number'},inplace = True)
company_basic.head()

,company_number,companies_house_company_profile.row_id,client_id,companies_house_company_profile.accounts.accounting_reference_date.day,companies_house_company_profile.accounts.accounting_reference_date.month,companies_house_company_profile.accounts.last_accounts.made_up_to,companies_house_company_profile.accounts.last_accounts.period_end_on,companies_house_company_profile.accounts.last_accounts.period_start_on,companies_house_company_profile.accounts.last_accounts.type,companies_house_company_profile.accounts.next_accounts.due_on,...,companies_house_company_officers.items[9].occupation,companies_house_company_officers.items[9].officer_role,companies_house_company_officers.items[9].resigned_on,companies_house_company_officers.items_per_page,companies_house_company_officers.kind,companies_house_company_officers.links.self,companies_house_company_officers.resigned_count,companies_house_company_officers.start_index,companies_house_company_officers.total_results,companies_house_company_officers.error
0,SC005364,0,0,30,6,2018-06-30,2018-06-30,2017-07-01,group,2019-12-31,...,Commercial Director,director,,35,officer-list,/company/SC005364/officers,23,0,33,


## Add Returned Company Name And Address To Input File

In [21]:
# Read in county & region(state) mapping data (sent with notebook) from filepath
county_code= pd.read_excel('https://s3.amazonaws.com/demyst-apis/demos/county_code.xlsx')
code = {}
for i in range(len(county_code)):
    code[county_code['county'][i].lower()] = county_code['parent'][i]

In [22]:
inputs['business_name'] = company_basic['companies_house_company_profile.company_name'].fillna('')
inputs['country'] = 'UK'
inputs['name'] = inputs['business_name']
inputs['search_term'] = inputs['business_name']
inputs['street'] = company_basic['companies_house_company_profile.registered_office_address.address_line_1'].fillna('')
inputs['city'] = company_basic['companies_house_company_profile.registered_office_address.locality'].fillna('')
inputs['post_code'] = company_basic['companies_house_company_profile.registered_office_address.postal_code'].fillna('')
#It needs to be ISO code
inputs['state'] = company_basic['companies_house_company_profile.registered_office_address.locality'].apply(lambda x: code[x.lower()] if x==x else '')



In [23]:
inputs[0:1]

,company_number,client_id,business_name,country,name,search_term,street,city,post_code,state
0,SC005364,0,Aberdeen Football Club Plc,UK,Aberdeen Football Club Plc,Aberdeen Football Club Plc,Pittodrie Stadium,Aberdeen,AB24 5QH,SCT


## Execute First Stage Of Appends - Vendors Which Use Company Name And Address As Input

In [24]:
providers = [
             'duedil_find_company',
             'acuris_business_search', 
             'globaldatabase_find_companies',
             'opencorporates_companies_search',
             'matchdeck_search',
             'orb_search'
             ]

In [26]:
first_results = analytics.enrich_and_download(providers, inputs[0:1], validate=False)
first_results.head()

,inputs.business_name,inputs.city,inputs.company_number,inputs.country,inputs.name,inputs.post_code,inputs.search_term,inputs.state,inputs.street,duedil_find_company.row_id,...,matchdeck_search.row_id,matchdeck_search.client_id,matchdeck_search.is_hit,matchdeck_search.total,matchdeck_search.error,orb_search.row_id,orb_search.client_id,orb_search.is_hit,orb_search.results_count,orb_search.error
0,Aberdeen Football Club Plc,Aberdeen,SC005364,UK,Aberdeen Football Club Plc,AB24 5QH,Aberdeen Football Club Plc,SCT,Pittodrie Stadium,0,...,0,0,False,0,,0,0,False,0,


In [27]:
#Merge the returned results against unique client ID
results = pd.merge(company_basic, first_results, how = 'left', left_on = 'client_id', right_on = 'duedil_find_company.client_id')
results.head()

,company_number,companies_house_company_profile.row_id,client_id,companies_house_company_profile.accounts.accounting_reference_date.day,companies_house_company_profile.accounts.accounting_reference_date.month,companies_house_company_profile.accounts.last_accounts.made_up_to,companies_house_company_profile.accounts.last_accounts.period_end_on,companies_house_company_profile.accounts.last_accounts.period_start_on,companies_house_company_profile.accounts.last_accounts.type,companies_house_company_profile.accounts.next_accounts.due_on,...,matchdeck_search.row_id,matchdeck_search.client_id,matchdeck_search.is_hit,matchdeck_search.total,matchdeck_search.error,orb_search.row_id,orb_search.client_id,orb_search.is_hit,orb_search.results_count,orb_search.error
0,SC005364,0,0,30,6,2018-06-30,2018-06-30,2017-07-01,group,2019-12-31,...,0,0,False,0,,0,0,False,0,


## Execute Second Stage Of Appends - Sources Which Require Other Inputs As Inputs

#### Run PIPL and review merged result

In [28]:
#Create PIPL input to enable matching against unique client ID
pipl_input = inputs[['client_id','country']].copy()
pipl_input['street'] = company_basic['companies_house_company_officers.items[0].address.address_line_1'].fillna('')
pipl_input['city'] = company_basic['companies_house_company_officers.items[0].address.locality'].fillna('')
pipl_input['post_code'] = company_basic['companies_house_company_officers.items[0].address.postal_code'].fillna('')
pipl_input['state'] = company_basic['companies_house_company_officers.items[0].address.locality'].fillna('').apply(lambda x:code[x.lower()] if x==x else '')
pipl_input['first_name'] = company_basic['companies_house_company_officers.items[0].name'].apply(lambda x: x.split()[0] if x==x else '')
pipl_input['last_name'] = company_basic['companies_house_company_officers.items[0].name'].apply(lambda x: x.split()[-1] if x==x else '')
pipl_input[0:1]

,client_id,country,street,city,post_code,state,first_name,last_name
0,0,UK,Pittodrie Stadium,Aberdeen,AB24 5QH,SCT,Roy,Johnston


In [29]:
#Run PIPL
pipl_results = analytics.enrich_and_download(['pipl_premium_search'], pipl_input[0:1], validate=False)
results = pd.merge(results, pipl_results, how = 'left',left_on = 'client_id', right_on = 'pipl_premium_search.client_id')

Verifying providers...
Starting enrichment...
Uploading data...


This enrichment will use 5.5 credits of the 998063059 credits your organization currently has.


Enrich Job ID: 6154


IntProgress(value=1, max=2)

Label(value='Checking status...')

In [30]:
results.head()

,company_number,companies_house_company_profile.row_id,client_id,companies_house_company_profile.accounts.accounting_reference_date.day,companies_house_company_profile.accounts.accounting_reference_date.month,companies_house_company_profile.accounts.last_accounts.made_up_to,companies_house_company_profile.accounts.last_accounts.period_end_on,companies_house_company_profile.accounts.last_accounts.period_start_on,companies_house_company_profile.accounts.last_accounts.type,companies_house_company_profile.accounts.next_accounts.due_on,...,pipl_premium_search.query.names[0].last_seen,pipl_premium_search.query.names[0].middle_name,pipl_premium_search.query.names[0].prefix,pipl_premium_search.query.names[0].suffix,pipl_premium_search.query.names[0].type,pipl_premium_search.query.names[0].valid_since,pipl_premium_search.query.search_pointer,pipl_premium_search.search_id,pipl_premium_search.visible_sources,pipl_premium_search.error
0,SC005364,0,0,30,6,2018-06-30,2018-06-30,2017-07-01,group,2019-12-31,...,,,,,,,,1905301337399845630943747278740181455,0,


#### Run DueDil and Review Merged Results

In [31]:
#Run DueDil
try:
    duedil_input = results[results['duedil_find_company.companies[0].company_id'].isna() == False][['duedil_find_company.client_id','duedil_find_company.companies[0].company_id']]
    duedil_input.rename(columns = {'duedil_find_company.client_id':'client_id',
                               'duedil_find_company.companies[0].company_id':'id'}, inplace = True)
    duedil_results = analytics.enrich_and_download(['duedil_company_vitals',
                                      'duedil_company_charges',
                                      'duedil_company_related_companies',
                                      'duedil_company_group_parents', 
                                      'duedil_company_group_subsidiaries', 
                                      'duedil_company_officers'], duedil_input, validate=False)
    results = pd.merge(results, duedil_results, how = 'left',left_on = 'client_id', right_on = 'duedil_company_vitals.client_id')
except:
    print('No company found for Duedil')

Verifying providers...
Starting enrichment...
Uploading data...


This enrichment will use 0.06 credits of the 998063054 credits your organization currently has.


Enrich Job ID: 6155


IntProgress(value=1, max=2)

Label(value='Checking status...')

In [32]:
results = pd.merge(results, duedil_results, how = 'left',left_on = 'client_id', right_on = 'duedil_company_vitals.client_id')
results.head()

,company_number,companies_house_company_profile.row_id,client_id,companies_house_company_profile.accounts.accounting_reference_date.day,companies_house_company_profile.accounts.accounting_reference_date.month,companies_house_company_profile.accounts.last_accounts.made_up_to,companies_house_company_profile.accounts.last_accounts.period_end_on,companies_house_company_profile.accounts.last_accounts.period_start_on,companies_house_company_profile.accounts.last_accounts.type,companies_house_company_profile.accounts.next_accounts.due_on,...,duedil_company_officers.officers[9].person.honorific_y,duedil_company_officers.officers[9].person.last_name_y,duedil_company_officers.officers[9].person.middle_name_y,duedil_company_officers.officers[9].person.nationalities[0].country_code_y,duedil_company_officers.officers[9].person.nationalities[0].demonym_y,duedil_company_officers.officers[9].type_y,duedil_company_officers.pagination.limit_y,duedil_company_officers.pagination.offset_y,duedil_company_officers.pagination.total_y,duedil_company_officers.error_y
0,SC005364,0,0,30,6,2018-06-30,2018-06-30,2017-07-01,group,2019-12-31,...,Mr,Yule,Kynoch,GB,British,person,10,0,30,


#### Run GlobalDatabase And Review Merged Results

In [33]:
#Run Global Database
try:
    globaldatabase_input = results[results['globaldatabase_find_companies.companies[0].id'].isna() == False][['globaldatabase_find_companies.client_id','globaldatabase_find_companies.companies[0].id']]
    globaldatabase_input.rename(columns = {'globaldatabase_find_companies.client_id':'client_id',
                               'globaldatabase_find_companies.companies[0].id':'id'}, inplace = True)
    globaldatabase_results = analytics.enrich_and_download(['globaldatabase_company_details'], globaldatabase_input, validate=False)
    results = pd.merge(results, globaldatabase_results, how = 'left',left_on = 'client_id', right_on = 'globaldatabase_company_details.client_id')
except:
    print('No company found for global database')

Verifying providers...
Starting enrichment...
Uploading data...


This enrichment will use 0.01 credits of the 998063054 credits your organization currently has.


Enrich Job ID: 6156


IntProgress(value=1, max=2)

Label(value='Checking status...')

In [34]:
results.head()

,company_number,companies_house_company_profile.row_id,client_id,companies_house_company_profile.accounts.accounting_reference_date.day,companies_house_company_profile.accounts.accounting_reference_date.month,companies_house_company_profile.accounts.last_accounts.made_up_to,companies_house_company_profile.accounts.last_accounts.period_end_on,companies_house_company_profile.accounts.last_accounts.period_start_on,companies_house_company_profile.accounts.last_accounts.type,companies_house_company_profile.accounts.next_accounts.due_on,...,globaldatabase_company_details.location,globaldatabase_company_details.phone,globaldatabase_company_details.post_code,globaldatabase_company_details.region,globaldatabase_company_details.registration_number,globaldatabase_company_details.size,globaldatabase_company_details.street,globaldatabase_company_details.vat_number,globaldatabase_company_details.website,globaldatabase_company_details.error
0,SC005364,0,0,30,6,2018-06-30,2018-06-30,2017-07-01,group,2019-12-31,...,PITTODRIE STREET,441224650400,AB24 5QH,Aberdeen,SC005364,,Pittodrie Stadium,,http://afc.co.uk,


#### Run Owler And Review Merged Results

In [35]:
#Run Owler
try:
    owler_input = results[results['globaldatabase_company_details.website'].isna() == False][['globaldatabase_company_details.client_id','globaldatabase_company_details.website']]
    owler_input.rename(columns = {'globaldatabase_company_details.client_id':'client_id',
                               'globaldatabase_company_details.website':'url'}, inplace = True)
    owler_results = analytics.enrich_and_download(['owler_company_premium'], owler_input, validate=False)
    results = pd.merge(results, owler_results, how = 'left',left_on = 'client_id', right_on = 'owler_company_premium.client_id')
except:
    print('No company found for Owler')

Verifying providers...
Starting enrichment...
Uploading data...


This enrichment will use 4.1 credits of the 998063054 credits your organization currently has.


Enrich Job ID: 6157


IntProgress(value=1, max=2)

Label(value='Checking status...')

In [36]:
results.head()

,company_number,companies_house_company_profile.row_id,client_id,companies_house_company_profile.accounts.accounting_reference_date.day,companies_house_company_profile.accounts.accounting_reference_date.month,companies_house_company_profile.accounts.last_accounts.made_up_to,companies_house_company_profile.accounts.last_accounts.period_end_on,companies_house_company_profile.accounts.last_accounts.period_start_on,companies_house_company_profile.accounts.last_accounts.type,companies_house_company_profile.accounts.next_accounts.due_on,...,owler_company_premium.perm_id,owler_company_premium.profile_url,owler_company_premium.revenue,owler_company_premium.short_name,owler_company_premium.stock.exchange,owler_company_premium.stock.ticker,owler_company_premium.twitter_link,owler_company_premium.website,owler_company_premium.youtube_link,owler_company_premium.error
0,SC005364,0,0,30,6,2018-06-30,2018-06-30,2017-07-01,group,2019-12-31,...,https://permid.org/1-4295897304,https://www.owler.com/iaApp/2298926/aberdeen-f...,,Aberdeen Football Club,,,http://twitter.com/AberdeenFC,http://www.afc.co.uk,,


#### Run Matchdeck And Review Merged Results

In [37]:
#Run Matchdeck
try:
    matchdeck_input = results[results['matchdeck_search.id'].isna() == False][['matchdeck_search.client_id','matchdeck_search.id']]
    matchdeck_input.rename(columns = {'matchdeck_search.client_id':'client_id',
                               'matchdeck_search.id':'id'}, inplace = True)
    matchdeck_results = analytics.enrich_and_download(['matchdeck_details'], matchdeck_input, validate=False)
    results = pd.merge(results, matchdeck_results, how = 'left',left_on = 'client_id', right_on = 'matchdeck_details.client_id')
except:
    print('No company found for matchdeck')

No company found for matchdeck


In [38]:
results.head()

,company_number,companies_house_company_profile.row_id,client_id,companies_house_company_profile.accounts.accounting_reference_date.day,companies_house_company_profile.accounts.accounting_reference_date.month,companies_house_company_profile.accounts.last_accounts.made_up_to,companies_house_company_profile.accounts.last_accounts.period_end_on,companies_house_company_profile.accounts.last_accounts.period_start_on,companies_house_company_profile.accounts.last_accounts.type,companies_house_company_profile.accounts.next_accounts.due_on,...,owler_company_premium.perm_id,owler_company_premium.profile_url,owler_company_premium.revenue,owler_company_premium.short_name,owler_company_premium.stock.exchange,owler_company_premium.stock.ticker,owler_company_premium.twitter_link,owler_company_premium.website,owler_company_premium.youtube_link,owler_company_premium.error
0,SC005364,0,0,30,6,2018-06-30,2018-06-30,2017-07-01,group,2019-12-31,...,https://permid.org/1-4295897304,https://www.owler.com/iaApp/2298926/aberdeen-f...,,Aberdeen Football Club,,,http://twitter.com/AberdeenFC,http://www.afc.co.uk,,


## Clean Output For Analysis - Remove Empty Columns, Errors, Row ID and Input Columns

In [39]:
results.drop(columns = [column for column in results.columns if 'error' in column or 'inputs' in column or 'row_id' in column or ('client_id' in column and column != 'client_id')], inplace = True)
results.replace([''], [None], inplace =True)
results.dropna(axis = 1, thresh = 1, inplace = True)

## Display Number of Columns Returned for Each Vendor

In [40]:
output_columns = sorted([column.split('.')[0] for column in results.columns if 'client_id' != column and 'company_number' != column])
vendor_count = Counter(output_columns)
for vendor in all_providers:
    if vendor not in vendor_count.keys():
        vendor_count[vendor] = 0
vendor_count

Counter({'acuris_business_search': 6603,
         'companies_house_company_officers': 430,
         'companies_house_company_profile': 43,
         'duedil_company_charges': 172,
         'duedil_company_group_parents': 12,
         'duedil_company_group_subsidiaries': 18,
         'duedil_company_officers': 438,
         'duedil_company_related_companies': 232,
         'duedil_company_vitals': 62,
         'duedil_find_company': 75,
         'globaldatabase_company_details': 12,
         'globaldatabase_find_companies': 5,
         'matchdeck_search': 2,
         'opencorporates_companies_search': 57,
         'orb_search': 2,
         'owler_company_premium': 17,
         'pipl_premium_search': 12,
         'matchdeck_details': 0})

## Display Final Results and Save to Local Folder

In [41]:
results.head()

,company_number,client_id,companies_house_company_profile.accounts.accounting_reference_date.day,companies_house_company_profile.accounts.accounting_reference_date.month,companies_house_company_profile.accounts.last_accounts.made_up_to,companies_house_company_profile.accounts.last_accounts.period_end_on,companies_house_company_profile.accounts.last_accounts.period_start_on,companies_house_company_profile.accounts.last_accounts.type,companies_house_company_profile.accounts.next_accounts.due_on,companies_house_company_profile.accounts.next_accounts.overdue,...,owler_company_premium.hq_address.street,owler_company_premium.industries[0].industry,owler_company_premium.is_hit,owler_company_premium.logo_url,owler_company_premium.name,owler_company_premium.perm_id,owler_company_premium.profile_url,owler_company_premium.short_name,owler_company_premium.twitter_link,owler_company_premium.website
0,SC005364,0,30,6,2018-06-30,2018-06-30,2017-07-01,group,2019-12-31,False,...,Pittodrie Street,"Travel, Recreation & Leisure",True,https://s3.amazonaws.com/owler-image/logo/aber...,Aberdeen Football Club Plc,https://permid.org/1-4295897304,https://www.owler.com/iaApp/2298926/aberdeen-f...,Aberdeen Football Club,http://twitter.com/AberdeenFC,http://www.afc.co.uk


In [42]:
results.to_csv('filename.csv', index = False)